In [45]:
%load_ext micropython_magic
%reload_ext micropython_magic

The micropython_magic extension is already loaded. To reload it, use:
  %reload_ext micropython_magic


In [46]:
%mpy -s {"/dev/cu.usbmodem_fs3_1"}

In [47]:
# %%micropython

import time
from config import config
from machine import RTC
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.time import ptp

NUM_SYNC_CYCLES = 15
TIMEOUT_MSEC = 5000

rtc = RTC()
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport.transport
miniot_network.connect()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Initial channel: {epn.wifi._sta.config('channel')}")

# Perform espnow scan to set the right channel
miniot_network.scan()
print(f"Final channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")

while True:
    # Perform periph sequence
    timestamps = ptp.sequence_periph(
        miniot_network,
        lambda miniot_msg: miniot_msg.msg,
        TIMEOUT_MSEC,
        initiate_sync=True,
        num_sync_cycles=NUM_SYNC_CYCLES)

    # Calculate offsets
    offsets = [ptp.calculate_offset(t1, t2, t3, t4) for t1, t2, t3, t4 in timestamps]

    print(f"timestamps: {timestamps}")
    print(f"offsets: {offsets}")

    # Process offsets
    ave_offset = ptp.process_offsets(offsets)

    # Update time
    rtc.offset(str(ave_offset).encode())

    print(f"ave_offset: {ave_offset}")
    print(f"ave_offset bytes: {str(ave_offset).encode()}")
    print(f"now: {rtc.datetime()}")

    time.sleep(15)


2000-01-01 00:05:08.308690 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:05:08.308692 WARNING-espnow-protocol:Peer has already been added, skipping. Peers: b"H'\xe2\xc7n\xfc"
2000-01-01 00:05:08.308694 DEBUG-espnow-protocol:espnow configured
2000-01-01 00:05:08.308695 INFO-miniot:Connecting MinIoT...
2000-01-01 00:05:08.308697 INFO-espnow-protocol:Connecting espnow...
Initial RTC datetime: (2000, 1, 1, 5, 0, 5, 8, 395206)
Initial RTC now: 308395790
My MAC: b'p\x04\x1d\xad|\xc0'
EPN Peer: b"H'\xe2\xc7n\xfc"
Initial channel: 11
2000-01-01 00:05:08.308702 DEBUG-espnow-protocol:Changing channel to: 1
2000-01-01 00:05:08.308706 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:05:08.308709 DEBUG-espnow-protocol:espnow configured
2000-01-01 00:05:08.308711 DEBUG-espnow-protocol:Sending SCAN REQ
2000-01-01 00:05:08.308790 DEBUG-espnow-protocol:Processing packet: <EspnowPacket object at 3c1b0d10>
2000-01-01 00:05:08.308792 DEBUG-espnow-protocol:Rx'ed SCAN RESP
Final channel: 1
